In [ ]:
#Fusion Model with Text and Images for Abnormality Detection
!pip install -q --upgrade keras-hub
!pip install -q --upgrade keras  # Upgrade to Keras 3.

In [ ]:
import os

#os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]

import json
import math
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras import losses
from keras import ops
from keras import optimizers
#from keras.optimizers import schedules
from keras import metrics
from keras.applications.imagenet_utils import decode_predictions
import keras_hub

# Import tensorflow for [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) and its preprocessing functions
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers
import tensorflow_hub as hub
import tensorflow.keras.layers
import tensorflow.keras.models
import tensorflow.keras.preprocessing.text
import tensorflow.keras.preprocessing.sequence
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D, Embedding, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
def plot_image_gallery(images, titles=None, num_cols=3, figsize=(6, 12)):
    num_images = len(images)
    print(num_images)
    images = np.asarray(images) / 255.0
    images = np.minimum(np.maximum(images, 0.0), 1.0)
    num_rows = (num_images + num_cols - 1) // num_cols
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize, squeeze=False)
    axes = axes.flatten()  # Flatten in case the axes is a 2D array

    for i, ax in enumerate(axes):
        if i < num_images:
            # Plot the image
            ax.imshow(images[i])
            ax.axis("off")  # Remove axis
            if titles and len(titles) > i:
                ax.set_title(titles[i], fontsize=12)
        else:
            # Turn off the axis for any empty subplot
            ax.axis("off")

    plt.show()
    plt.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def load_data(data_dir):
  image_files = []
  labels = []
  for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    print(class_name)
    if os.path.isdir(class_dir):
      for filename in os.listdir(class_dir):
        file_path = os.path.join(class_dir, filename)
        if filename.endswith(".jpg"):
          image_files.append(file_path)
          if(class_name == "Abnormal"):
            labels.append(1)
          else:
            labels.append(0)
          #labels.append(class_name)
        #elif filename.endswith(".txt"):
          #print(file_path)
          #text_files.append(file_path)
          #labels.append(class_name)

  return image_files, labels

def load_text(image_files):

  text_files = []
  for path in image_files:
    #image = tf.io.read_file(image_path)
    #img_count += 1
    #image = tf.image.decode_jpeg(image, channels=3)
    #image = tf.image.resize(image, [224, 224])  # Resize as needed
    #image = tf.cast(image, tf.float32) / 255.0  # Normalize
    text_path = path.replace(".jpg", ".txt")
    text_files.append(text_path)
  #print(text_path)
  return text_files

def preprocess_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [224, 224])  # Resize as needed
  image = tf.cast(image, tf.float32) / 255.0  # Normalize
  #text = text.to_tensor(default_value="UNK")  # Handle empty sequences
  return image

def preprocess_text(text_path):
  text = tf.io.read_file(text_path)
  return text



img_count = 0
text_count = 0
# Load and preprocess data
image_files, labels = load_data("/content/drive/MyDrive/CS230_Project/Dental_DB/Classify/Data/")
text_files = load_text(image_files)


image_dataset = tf.data.Dataset.from_tensor_slices(image_files)
image_dataset = image_dataset.map(preprocess_image)

text_dataset = tf.data.Dataset.from_tensor_slices(text_files)
text_dataset = text_dataset.map(preprocess_text)
text_processor = layers.TextVectorization(
    max_tokens=20000,
    output_sequence_length=100  # Pad sequences to a maximum length of 100
)
text_processor.adapt(text_dataset.map(lambda text: text))

label_dataset = tf.data.Dataset.from_tensor_slices(labels)


# Combine datasets
dataset = tf.data.Dataset.zip((image_dataset, text_dataset), label_dataset)

def preprocess_data(inputs, label):
    image, text = inputs
    processed_text = text_processor(text)
    return (image, processed_text), label  # Return as ((image, text), label)

def get_test_data(inputs, label):
    image, text = inputs
    #processed_text = text_processor(text)
    return image, label  # Return as ((image, text), label)



# Calculate the split sizes
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = int(0.1 * len(dataset))

# Shuffle the dataset
dataset = dataset.shuffle(buffer_size=len(dataset))

# Split the dataset
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
test_dataset = dataset.skip(train_size + val_size).take(test_size)

train_dataset = train_dataset.map(preprocess_data)
val_dataset = val_dataset.map(preprocess_data)
test_dataset = test_dataset.map(get_test_data)
print(len(test_dataset))
def format_data(inputs, label):
    # Create a dictionary with two labels, one for each output layer
    image, text = inputs
    return (image, text), (label, label)

# Apply the formatting function to your datasets
train_dataset = train_dataset.map(format_data)
val_dataset = val_dataset.map(format_data)
#for sample in test_dataset.take(1):
  #print(sample)
  #print(sample[0].shape)
#val_dataset = val_t_dataset.map(get_test_data)
#test_dataset = full_dataset.skip(train_size + val_size)
#test_dataset = test_dataset.map(preprocess_data)

print("Number of images:", img_count)
print("Number of texts:", text_count)
# Apply the combine_data function to your datasets
#train_dataset = train_dataset.map(lambda image, text, label: combine_data(image, text, label))
#val_dataset = val_dataset.map(lambda image, text, label: combine_data(image, text, label))
#val_dataset = test_dataset.skip(val_size)
#test_dataset = test_dataset.take(test_size)

Abnormal
Normal
100
Number of images: 0
Number of texts: 0


In [ ]:
train_dataset = train_dataset.batch(32)  # Adjust batch size as needed
val_dataset = val_dataset.batch(32)
test_dataset = test_dataset.batch(32)

In [ ]:
#def create_image_encoder():
 # #image_input = tf.keras.Input(shape=(224, 224, 3), name='image')
  #base_model = ResNet50(weights='imagenet', include_top=False)(image_input)
 # image_features = tf.keras.layers.GlobalAveragePooling2D()(base_model)
 # return Model(inputs=image_input, outputs=image_features, name = "Image_Encoder")

In [ ]:
def create_image_encoder():
  image_input = tf.keras.Input(shape=(224, 224, 3), name='image')
  base_model = keras.applications.ResNet50(weights='imagenet', include_top=False)(image_input)
  image_features = tf.keras.layers.GlobalAveragePooling2D()(base_model)


  image_features = tf.keras.layers.Dense(256, activation="relu")(image_features)
  image_features = tf.keras.layers.Dropout(0.3)(image_features)
  image_features = tf.keras.layers.Dense(128, activation="relu")(image_features)
  image_features = tf.keras.layers.Dropout(0.3)(image_features)
  image_features = tf.keras.layers.Dense(64, activation="relu")(image_features)
  image_features = tf.keras.layers.Dropout(0.3)(image_features)
  #output = tf.keras.layers.Dense(2, activation='softmax', name = "main_output")(image_features)
  return Model(inputs=image_input, outputs=image_features, name = "Image_Encoder")

In [ ]:
#def create_image_text_model():

text_input = tf.keras.Input(shape=(None,), name='text')

# Image processing layers
image_encoder = create_image_encoder()
image_input = image_encoder.input
image_features = image_encoder.output

# Text processing layers
text_features = tf.keras.layers.Embedding(10000, 64)(text_input)
text_features = tf.keras.layers.LSTM(64)(text_features)

# Concatenate image and text features
combined_features = tf.keras.layers.concatenate([image_features, text_features])
combined_features = tf.keras.layers.Dense(64, activation='relu')(combined_features)
compiled_features = tf.keras.layers.Dropout(0.3)(combined_features)
# Output layer
main_output = tf.keras.layers.Dense(1, activation='sigmoid', name = "main_output")(combined_features)

aux_output = tf.keras.layers.Dense(1, activation='sigmoid', name='image_only_output')(image_features)

# Create the model

model = tf.keras.Model(inputs=[image_input, text_input], outputs=[main_output, aux_output])
    #model._layers.append(image_encoder)  # Append model to _layers list.
    #model._layers[-1]._name = "Image_Encoder" # Assign layer the desired name.
    #return model

#model = create_image_text_model()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.compile(optimizer='adam', loss={ "main_output": "binary_crossentropy",
        "image_only_output": "binary_crossentropy",
    }, loss_weights={"main_output": 1.0, "image_only_output": 0.7}, metrics={'main_output': ['accuracy'], 'image_only_output': ['accuracy']}) # Provide a dictionary of metrics specifying metrics

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)        │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 7, 7, 2048)     │     23,587,712 │ image[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d  │ (None, 2048)           │              0 │ resnet50[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │        524,544 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 256)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │         32,896 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ text (InputLayer)         │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │          8,256 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 64)       │        640,000 │ text[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 64)             │         33,024 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 128)            │              0 │ dropout_2[0][0],       │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │          8,256 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ main_output (Dense)       │ (None, 1)              │             65 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ image_only_output (Dense) │ (None, 1)              │             65 │ dropout_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 24,834,818 (94.74 MB)

 Trainable params: 24,781,698 (94.53 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:

history = model.fit(train_dataset, epochs=20, validation_data=val_dataset)


In [ ]:
plt.plot(history.history['val_image_only_output_accuracy'])
plt.plot(history.history['val_main_output_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Image and Text', 'Image only'], loc='upper left')
plt.show()

In [ ]:
image_only_model = Model(inputs=image_input, outputs=aux_output)
image_only_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Get predictions
y_pred_prob = image_only_model.predict(test_dataset)

# Apply threshold
y_pred = np.where(y_pred_prob > 0.98, 1, 0)  # Adjust 0.6 to your desired threshold

# Calculate metrics

def custom_accuracy(y_true, y_pred):
    threshold = 0.99
    y_pred = tf.where(y_pred > threshold, 1.0, 0.0)
    return tf.keras.metrics.binary_accuracy(y_true, y_pred)
image_only_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', custom_accuracy])
image_only_model.evaluate(test_dataset)

In [ ]:
image_only_model = Model(inputs=image_input, outputs=aux_output)
image_only_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
image_only_model.evaluate(test_dataset)